# Imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix

from tqdm import tqdm
from time import time

In [2]:
X_train = np.load('data/x_train.npy')
y_train = np.load('data/y_train.npy')

X_test = np.load('data/x_test.npy')
y_test = np.load('data/y_test.npy')

#X_val = np.load('data/x_val.npy')
#y_val = np.load('data/y_val.npy')

In [3]:
#X_train = np.concatenate((X_train, X_val), axis=0)
#y_train = np.concatenate((y_train, y_val), axis=0)

# Playground

In [4]:
np.unique(y_train, return_counts=True)

(array([0, 1, 2, 3, 4]), array([   166,    210,  62760,   6637, 885235]))

4 is normal class

In [5]:
normal_data = X_train[y_train == 4] # 11 is normal class

# Helpers

In [6]:
def evaluate_model(model, y_test, y_pred):
    positive_class = 'normal'
    
    accuracy = accuracy_score(y_test, y_pred)
    
    precision = precision_score(y_test, y_pred, pos_label=positive_class)
    recall = recall_score(y_test, y_pred, pos_label=positive_class)
    f1 = f1_score(y_test, y_pred, pos_label=positive_class)
    
    metrics = {
        'model': [model],
        'accuracy': [accuracy],
        'precision': [precision],
        'recall': [recall],
        'f1': [f1],
    }
    
    return metrics

# OCSVM

In [7]:
from sklearn.svm import OneClassSVM

In [8]:
ocsvm = OneClassSVM(kernel='rbf', gamma=5, nu=0.01)

batch_size = 50000
n_batches = 1# int(np.ceil(len(normal_data) / batch_size))

for i in tqdm(range(n_batches), desc="Training OCSVM"):
    start = i * batch_size
    end = min((i + 1) * batch_size, len(normal_data))
    batch_data = normal_data[start:end]
    
    if i == 0:
        ocsvm.fit(batch_data)
    else:
        ocsvm.fit(np.vstack((ocsvm.support_vectors_, batch_data)))

Training OCSVM: 100%|██████████| 1/1 [00:02<00:00,  2.02s/it]


In [9]:
y_pred_test_ocsvm = ocsvm.predict(X_test)
y_pred_test_ocsvm = np.where(y_pred_test_ocsvm == 1, 'normal', 'anomaly')
y_test_converted_ocsvm = np.where(y_test == 4, 'normal', 'anomaly')

In [10]:
ocsvm_df = pd.DataFrame(evaluate_model("ocsvm", y_test_converted_ocsvm, y_pred_test_ocsvm)) 
ocsvm_df

,model,accuracy,precision,recall,f1
0,ocsvm,0.990509,0.999736,0.990032,0.99486


# LOF

In [11]:
from sklearn.neighbors import LocalOutlierFactor

In [12]:
lof_start = time()

lof = LocalOutlierFactor(n_neighbors=50, contamination=0.0001, novelty=True)
lof.fit(X_train)

print(f"seconds: {time() - lof_start}")

seconds: 439.2442111968994


In [13]:
y_pred_test_lof = lof.predict(X_test)
y_pred_test_lof = np.where(y_pred_test_lof == 1, 'normal', 'anomaly')
y_test_converted_lof = np.where(y_test == 4, 'normal', 'anomaly')

In [14]:
lof_df = pd.DataFrame(evaluate_model("lof", y_test_converted_lof, y_pred_test_lof))  
lof_df

,model,accuracy,precision,recall,f1
0,lof,0.927804,0.927839,0.999955,0.962548


# IF

In [15]:
from sklearn.ensemble import IsolationForest

In [52]:
if_start = time()

iforest = IsolationForest(n_estimators=100, max_samples=256, contamination=0.1, random_state=42)
iforest.fit(X_train)

print(f"seconds: {time() - if_start}")

seconds: 43.22564077377319


In [53]:
y_pred_test_if = iforest.predict(X_test)
y_pred_test_if = np.where(y_pred_test_if == 1, 'normal', 'anomaly')
y_test_converted_if = np.where(y_test == 4, 'normal', 'anomaly')

In [54]:
if_df = pd.DataFrame(evaluate_model("if", y_test_converted_if, y_pred_test_if))  
if_df

,model,accuracy,precision,recall,f1
0,if,0.973022,1.0,0.970923,0.985247


# Results

In [55]:
results_df = pd.concat([ocsvm_df, lof_df, if_df], ignore_index=True)
results_df.to_csv('results/results_wustl.csv', index=False)

results_df

,model,accuracy,precision,recall,f1
0,ocsvm,0.990509,0.999736,0.990032,0.994860
1,lof,0.927804,0.927839,0.999955,0.962548
2,if,0.973022,1.000000,0.970923,0.985247
